# Output Parsers  
**It is in langchain help convert raw LLM responses into structured formats like JSON,CSV,Pydantic. They ensure consistency, validation, and ease of use in applications.  
It work with both LLM models which in can and can not give the structured output**

# String Output parser

In [11]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
import os
load_dotenv()
GROQ_API_KEY=os.environ.get("GROQ_API_KEY")

llm=ChatGroq(model="llama-3.3-70b-versatile")

In [12]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate

# llm = HuggingFaceEndpoint(
#     repo_id="google/gemma-2-2b-it",
#     task="text-generation"
# )


# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

prompt1 = template1.invoke({'topic':'black hole'})

result = llm.invoke(prompt1)

prompt2 = template2.invoke({'text':result.content})

result1 = llm.invoke(prompt2)

print(result1.content)


A black hole is a region in space where gravity is so strong that nothing can escape, formed when a massive star collapses. Black holes have unique properties, including mass, event horizon, singularity, and gravitational pull. There are four types of black holes, each with different characteristics, and their presence can be inferred through observational evidence such as X-rays and star motions. The study of black holes has led to significant advances in understanding spacetime and gravity, but many challenges and open questions remain. Continued research and observation are helping to shed light on the nature of black holes and their role in the universe.


In [19]:
from langchain_core.output_parsers import StrOutputParser
# 1st prompt -> detailed report
template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

# 2nd prompt -> summary
template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

chain = template1 | llm | parser | template2 | llm | parser

result = chain.invoke({'topic':'black hole'})

print(result)

A black hole is a region in space with an intense gravitational pull that nothing, including light, can escape. Black holes are formed when massive stars collapse, creating a boundary called the event horizon, beyond which anything that enters cannot escape. There are four types of black holes, including stellar, supermassive, intermediate-mass, and primordial black holes, each with distinct properties and formation mechanisms. The existence of black holes has been confirmed through various observational evidence, including X-rays, gamma rays, and gravitational waves. Recent discoveries, such as the first image of a black hole, have significantly advanced our understanding of these mysterious objects and their role in the universe.


# JSON string output parser


In [13]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = template | llm | parser

result = chain.invoke({'topic':'black hole'})

print(result)

{'facts': [{'id': 1, 'fact': 'A black hole is a region in space where the gravitational pull is so strong that nothing, including light, can escape.'}, {'id': 2, 'fact': 'Black holes are formed when a massive star collapses in on itself and its gravity becomes so strong that it warps the fabric of spacetime around it.'}, {'id': 3, 'fact': 'There are four types of black holes, each with different properties and formation mechanisms: stellar black holes, intermediate-mass black holes, supermassive black holes, and miniature black holes.'}, {'id': 4, 'fact': "The point of no return around a black hole is called the event horizon, and once something crosses the event horizon, it is trapped by the black hole's gravity and cannot escape."}, {'id': 5, 'fact': 'Black holes can distort and bend light around them, a phenomenon known as gravitational lensing, which allows scientists to study them indirectly and learn more about their properties.'}]}


# Pydanic

In [16]:

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Person(BaseModel):

    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | llm | parser

final_result = chain.invoke({'place':'sri lankan'})

print(final_result)

name='Nethmi Fernando' age=25 city='Colombo'


# StructuredOuptutParser

In [18]:
from langchain.output_parsers import StructuredOutputParser,ResponseSchema
schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | llm | parser

result = chain.invoke({'topic':'black hole'})

print(result)

{'fact_1': 'Black holes are regions in space where the gravitational pull is so strong that nothing, including light, can escape.', 'fact_2': 'Black holes are formed when a massive star collapses in on itself and its gravity becomes so strong that it warps the fabric of spacetime around it.', 'fact_3': 'There are four types of black holes, including stellar black holes, intermediate-mass black holes, supermassive black holes, and miniature black holes, each with different properties and formation mechanisms.'}
